# Installations and Imports

In [10]:
#In case you don't have qiskit, install it now
%pip install qiskit --quiet
%pip install qiskit-aer --quiet

In [11]:
#Installing/upgrading pylatexenc seems to have fixed my mpl issue
#If you try this and it doesn't work, try also restarting the runtime/kernel
%pip install pylatexenc --quiet

In [12]:
#Let's go ahead and import all this stuff too
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit.visualization import plot_histogram, plot_bloch_multivector

# Quantum Error Correction

In [16]:
import qiskit
import numpy as np

In [24]:
def rand(a=0, b=1):
    return a + (b-a)*np.random.random()

## Encoding and Decoding

In [18]:
def encode(qc, qx):
    # Encode qubit qx[0].

    qc.h(qx[1])
    qc.h(qx[2])
    qc.cz(qx[1], qx[2])
    qc.cx(qx[1], qx[0])
    qc.cx(qx[2], qx[0])

    return

In [19]:
def decode(qc, qx):
    # Decode qubit qx[0].

    qc.cx(qx[2], qx[0])
    qc.cx(qx[1], qx[0])
    qc.cz(qx[1], qx[2])
    qc.h(qx[2])
    qc.h(qx[1])

    return

## Error Model

In [28]:
def error_model(qc, qx, e=None, k=None):
    # Apply a random single-qubit error to one of the encoding qubits
    # DO NOT MODIFY!

    n = len(qx)

    if (e is None):
        e = np.random.randint(0,5) # 0 = no error, 1 = X error, 2 = Y error, 3 = Z error, 4 = random unitary
    if (k is None):
        k = np.random.randint(0,n) # index of qubit on which error occurs

    if e == 0:
        print("no error")
        pass # no error - do nothing
    elif e == 1:
        print("X error on qubit " + str(k))
        qc.x(qx[k]) # bit-flip error on qubit k
    elif e == 2:
        print("Y error on qubit " + str(k))
        qc.y(qx[k]) # bit-flip & phase error on qubit k
    elif e == 3:
        print("Z error on qubit " + str(k))
        qc.z(qx[k]) # phase error on qubit k
    else:
        print("Random unitary on qubit " + str(k))
        theta = np.arccos(2*rand()-1)
        phi = 2*np.pi*rand()
        lamb = 2*np.pi*rand()
        qc.u(theta, phi, lamb, qx[k])

## Syndrome Measurement

In [21]:
def syndrome(qc, qx, qs):
    # Apply gates to "measure" the syndromes

    qc.h(qs[0])
    qc.h(qs[1])

    qc.cx(qs[0],qx[0])
    qc.cx(qs[0],qx[1])
    qc.cz(qs[0],qx[2])

    qc.cx(qs[1],qx[0])
    qc.cz(qs[1],qx[1])
    qc.cx(qs[1],qx[2])

    qc.h(qs[0])
    qc.h(qs[1])

    return

## Error Correction

In [22]:
def qec_conditionals(qc, qx, qs, cs):
    # Correct errors using measured syndromes and conditional gates
    # check what commutes to find errors
    qc.measure(qs[0],cs[0])
    qc.measure(qs[1],cs[1])

    qc.x(qx[2]).c_if(cs, 1)
    qc.z(qx[2]).c_if(cs, 2)
    qc.y(qx[2]).c_if(cs, 3)


    return

## Main Code Block

In [29]:
n = 3 # number of physical qubits
k = 1 # number of logical qubits

for i in range(5):

    # prepare the quantum circuit
    qx = qiskit.QuantumRegister(n)
    qs = qiskit.QuantumRegister(n-k)
    cx = qiskit.ClassicalRegister(n)
    cs = qiskit.ClassicalRegister(n-k)
    qc = qiskit.QuantumCircuit(qx, qs, cx, cs)

    # prepare a random single-qubit state in qx[0]
    theta = np.arccos(2*rand()-1)
    phi = 2*np.pi*rand()
    lamb = 0
    qc.u(theta, phi, lamb, qx[0])

    # encode the qubit
    encode(qc, qx)

    # apply a random error to one of the encoding qubits
    error_model(qc, qx, i, 2) # error on qubit 2

    # apply gates to perform a syndrome measurement
    syndrome(qc, qx, qs)

    # detect and correct error using the syndromes
    qec_conditionals(qc, qx, qs, cs)

    # decode the corrected, encoded qubit
    decode(qc, qx)

    # prepare qubit qx[0] for measurement
    qc.u(theta, np.pi-lamb, np.pi-phi, qx[0])

    # measure the qx register
    for i in range(len(qx)):
        qc.measure(qx[i], cx[i])

    # measure the qs register
    for j in range(len(qs)):
        qc.measure(qs[j], cs[j])

    # execute the quantum circuit
    backend=qiskit.Aer.get_backend('qasm_simulator')
    job = qiskit.execute(qc, backend, shots=1024)
    data = job.result().get_counts(qc)
    print(data)

no error
{'00 000': 1024}
X error on qubit 2
{'01 000': 1024}
Y error on qubit 2
{'11 000': 1024}
Z error on qubit 2
{'10 000': 1024}
Random unitary on qubit 2
{'00 000': 1, '10 000': 7, '11 000': 158, '01 000': 858}


## My own scratch work

In [30]:
c = ClassicalRegister(6)
q1 = QuantumRegister(4)
prob1 = QuantumCircuit(q1,c)
prob1.h(0)
prob1.h(1)
prob1.h(2)
prob1.h(3)
prob1.t(2)
prob1.t(3)
prob1.cx(0,2)
prob1.cx(1,3)

prob1.()
prob1.cs()

prob1.measure(0,2)
prob1.measure(1,3)
prob1.measure(2,4)
prob1.measure(3,5)

backend =  qiskit.Aer.get_backend('qasm_simulator')
job = qiskit.execute(prob1, backend, shots=1024)
data = job.result().get_counts(prob1)
print(data)

{'101110': 48, '010001': 60, '110111': 82, '011101': 65, '010101': 57, '100010': 63, '000000': 65, '100110': 53, '000100': 69, '110011': 68, '111011': 57, '001000': 58, '001100': 74, '101010': 72, '111111': 61, '011001': 72}
